In [1]:
# Importing the basic libraries we will require for the project

# Libraries to help with reading and manipulating data
import pandas as pd
import multiprocessing
import os


#importing global
import glob

from itertools import repeat


# Code to ignore warnings from function usage
import warnings;
warnings.filterwarnings('ignore')

# Concatenation of meteorological variables


In the next section historicals sensors measurement for each meteorological variable data will be concatenated in one big "csv" file. The data is originally stored in ".data" files. A ".data" file is a separate file by "
The file is namened by as follows:BSHG_TT_D@10000000.data
- BSHG_TT_D, corresponds of the variable name tag
- 10000000, code station

In [140]:
#concatenating all the files in a list. One variable at a time
path ='/Users/yourdatapath'
file_list = glob.glob(path+"/*.data")
# list of .data files we want to merge.
data_list = []
station_list = []
for file in file_list:
    _df=pd.read_csv(file, sep="|")
    data_list.append(_df)
    station_list.extend(repeat(int(os.path.basename(file).split('/')[-1].split('@')[-1].split('.')[0]), len(_df)))

In [131]:
#visualizing the list
#data usta is a list of three data per row
print(data_list)

In [132]:
#Confirm the len of the lists 
print({'station len':len(station_list), 'data len': len(data_list)}) 

In this step the code_station list will be stored as a "dataframe" and then concatenated with data_list. Repeat this step for each folder, for each variable

In [43]:
codeStation = pd.DataFrame({'Codigo': station_list})
metereological_var = pd.concat([pd.concat(data_list, axis=0, ignore_index=True), codeStation], axis=1)
#saving the file in csv format
metereological_var.to_csv('/yourpath/metereological_var_all.csv',index=False)

## Making match of the code station

In this section the measurement of each code station will be asigned to its region, department, altitude, longitude and latitude


In [2]:
#reading the data file of "national code of stations"
data_station= pd.read_excel('datapath/CNE_IDEAM.xls')

In [3]:
#filtering the desired variables
dataStation_filtered= data_station[['CODIGO', 'altitud', 'longitud','latitud', 'DEPARTAMENTO']]

### Standarizing the names

In [4]:
# Define a mapping dictionary for replacements
replacement_mapping = {
    'Caqueta': 'Caquetá',
    'RIsaralda': 'Risaralda',
    'Atlantico': 'Atlántico',
    'Vaupes': 'Vaupés',
    'Choco': 'Chocó',
    'Antioquía': 'Antioquia',
    'Guania': 'Guainía',
    'Norte de santander': 'Norte de Santander'
}

# Apply the replacements in a single pass
dataStation_filtered['DEPARTAMENTO'] = dataStation_filtered['DEPARTAMENTO'].replace(replacement_mapping)


In [5]:
#making a sample of the data, to check the content
dataStation_filtered.sample(5, random_state=0)

,CODIGO,altitud,longitud,latitud,DEPARTAMENTO
2445,25020230,170,-73.339472,9.562167,Cesar
2031,26170150,1819,-75.838806,5.791056,Antioquia
3231,23177020,75,-73.983333,7.166667,Antioquia
1888,52010150,2450,-76.588972,2.292750,Cauca
1867,33055010,167,-69.803333,4.494722,Vichada


In [6]:
#checking for null values
dataStation_filtered.isnull().sum()

CODIGO          0
altitud         0
longitud        0
latitud         0
DEPARTAMENTO    0
dtype: int64

### Categorizing regions based on the 'DEPARTAMENTO'

In [7]:
#Assgning to each department the region
region_caribe = ['Atlántico', 'Bolívar', 'Cesar', 'Magdalena', 'La Guajira', 'Sucre', 'Córdoba']
region_andina = ['Bogotá','Antioquia', 'Caldas', 'Norte de Santander', 'Quindío', 'Risaralda', 'Santander', 'Boyacá', 'Cundinamarca', 'Tolima', 'Huila']
region_amazonas = ['Amazonas', 'Caquetá', 'Guainía', 'Putumayo', 'Guaviare', 'Vaupés']
region_pacifica = [ 'Cauca', 'Chocó', 'Nariño', 'Valle del Cauca']
region_orinoquia = ['Arauca', 'Casanare', 'Meta', 'Vichada']
providencia = ['Archipiélago de San Andres, Providencia y Santa Catalina']

In [8]:
#funtion to make match
def matchDepartment(x):
    if x in region_amazonas :
        return 'Amazonas'
    elif x in region_andina :
        return 'Andina'
    elif x in region_caribe:
        return 'Caribe'         
    elif x in region_pacifica:
        return 'Pacifica'
    elif x in region_orinoquia:
        return 'Orinoquía'
    elif x in providencia:
        return 'Insular'

In [9]:
dataStation_filtered["Region"] = dataStation_filtered['DEPARTAMENTO'].apply(matchDepartment)

In [10]:
#checking for null values
dataStation_filtered.isnull().sum()

CODIGO          0
altitud         0
longitud        0
latitud         0
DEPARTAMENTO    0
Region          0
dtype: int64

In [ ]:
#reaming "DEPARTAMENTO","CODIGO" to lowercase
dataStation_filtered=dataStation_filtered.rename(columns={'DEPARTAMENTO':'Departamento', 'CODIGO':'Codigo'})

In [41]:
#saving sations code
dataStation_filtered.to_csv('yourdatapatha/dataStation_filtered.csv',index=False)

### Reading the data to add to each stations its geographicals coordinates

In [ ]:
# Define paths for input and output directories
input_directory = 'yourpath/'
output_directory = 'yourdatapath/'
# Define a list of variables
variables = ['brillosolar', 'nubosidad', 'velviento', 'temperatura', 'dirviento', 'rocio','humedadrelativa', 
             'fetatmosferico', 'evaporacion', 'precipitacion', 'recorridoviento', 'tensionvapor']

In [ ]:
# Read the stations data
stations = pd.read_csv('yourdatapatha/dataStation_filtered.csv')
stations = stations[stations['Codigo'].notnull()].set_index('Codigo')

In [ ]:
#iterate over each variable
for var in variables:
    input_file_path = os.path.join(input_directory, f'{var}_all.csv')
    output_file_path = os.path.join(output_directory, f'{var}_location.csv')

    # Read the CSV file with datetime parsing
    df = pd.read_csv(input_file_path, parse_dates=['Datetime'])

    # Filter datetime data from the year 2012
    df = df[df['Datetime'].dt.year >= 2012]

    # Filter rows with non-null 'Codigo' values
    df = df[df['Codigo'].notnull()].set_index('Codigo')

    # Join with stations data
    df = stations.join(df, how='right')

    # Save the result to a CSV file
    df.to_csv(output_file_path, index=False)

    # Clear the dataframe to release memory
    del df
    